In [7]:
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 

os.chdir("/Users/tienphat/Documents/HCMUS/Statistic_Application/covid_analysis")

data_dir = "dataset/csse_covid_19_daily_reports_us"
list_dir = os.listdir(data_dir)

In [14]:
# FILL na with zero

import tqdm
cols_to_fillna=['Confirmed', 'Deaths', 'Recovered', 'Active']

for filename in tqdm.tqdm(list_dir):
    if not filename.endswith('.csv'):
        continue
    filedir = os.path.join(data_dir, filename)
    file_df = pd.read_csv(filedir)
    file_df[cols_to_fillna] = file_df[cols_to_fillna].fillna(value=0)
    file_df.to_csv(filedir, index=False)


100%|██████████| 111/111 [00:00<00:00, 165.23it/s]


In [37]:
# Group files by state

import tqdm 

sample_file = os.path.join(data_dir, list_dir[0])
sample_df = pd.read_csv(sample_file)
list_states = sample_df['Province_State']
imp_columns = ['Day', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'People_Tested']

combine_dict = dict()

# Sort day string
from datetime import datetime
list_day = [s.split('.')[0] for s in list_dir if s.endswith('.csv')]
sorted_list_day = sorted(
    list_day,
    key=lambda x: datetime.strptime(x, '%m-%d-%Y'), reverse=False
)

for day in tqdm.tqdm(sorted_list_day):
    day_filedir = day+'.csv'
    day_df = pd.read_csv(os.path.join(data_dir, day_filedir))
    
    for state in list_states:
        if state not in combine_dict.keys():
            combine_dict[state] = pd.DataFrame(columns=imp_columns)

        state_df = day_df[day_df['Province_State']==state]

        append_dict = {'Day': day}
        for col in imp_columns[1:]:
            append_dict.update({col: state_df[col].values[0]})
        
        combine_dict[state] = combine_dict[state].append(append_dict, ignore_index=True)

for key in combine_dict.keys():
    combine_dict[key].to_csv(os.path.join('dataset/csse_combine_state', key+'.csv'), index=False)


100%|██████████| 108/108 [00:31<00:00,  3.45it/s]
